In [137]:
import numpy as np
from numpy import cos, sin, pi
import matplotlib.pyplot as plt
from libs.XY_model import XYSystem
import cv2
from tqdm import tqdm

In [138]:
def get_curl(x, neighbors):    
    N = x.shape[0]
    curl = np.zeros([N, N])
    ux = cos(x)
    uy = sin(x)
    for i in range(N):
        for j in range(N):
            for a in range(1, neighbors):
                # curl[i, j] += (  ( (2 * sin(x[i, j])) - sin(x[(i+a)%N, j]) - sin(x[(i-a)%N, j]) ) -
                #                  ( (2 * cos(x[i, j])) - cos(x[i, (j+a)%N]) - cos(x[i, (j-a)%N]) )   )/a
                
                curl[i, j] += (  ( uy[i, j] - uy[(i-a)%N, j] ) -
                                 ( ux[i, j] - ux[i, (j-a)%N] ) )/a
                
                
                # curl[i, j] += (  ( (2 * sin(x[i, j])) - sin(x[(i+a)%N, (j+a)%N]) - sin(x[(i-a)%N, (j-a)%N]) ) -
                #                  ( (2 * cos(x[i, j])) - cos(x[(i+a)%N, (j+a)%N]) - cos(x[(i-a)%N, (j-a)%N]) )   )/(a * (2**0.5))
                
                # curl[i, j] += (  ( (2 * sin(x[i, j])) - sin(x[(i+a)%N, (j-a)%N]) - sin(x[(i-a)%N, (j+a)%N]) ) -
                #                  ( (2 * cos(x[i, j])) - cos(x[(i+a)%N, (j-a)%N]) - cos(x[(i-a)%N, (j+a)%N]) )   )/(a * (2**0.5))

             
    curl /= neighbors
    return curl

In [139]:
def list2matrix(S):
        N=int(np.size(S))
        L = int(np.sqrt(N))
        S=np.reshape(S,(L,L))
        return S

In [140]:
def plot_heatmap(spin_config, curl, temp, name = 'result', heat = True, just_heat = False, spin = False):
    N = spin_config.shape[0]
    X, Y = np.meshgrid(np.arange(0,N),np.arange(0, N))
    U = np.cos(spin_config)
    V = np.sin(spin_config)
    plt.figure(figsize=(4,4),dpi=800, facecolor='white')
    teta = np.arccos(V)
    for i in range(V.shape[0]):
        for j in range(V.shape[1]):
            if(V[i, j] < 0):
                teta[i, j] += pi
    if(heat):
        plt.imshow(curl, extent=(0, N, 0, N))
        plt.colorbar()
    if(just_heat):
        plt.imshow(curl,extent=(0, N, 0, N))
        plt.colorbar()
        plt.savefig(f'./images/{name}.png', dpi = 800)
        plt.close()
        return
    if(spin):
        plt.imshow(spin_config, extent=(0, N, 0, N))
        plt.colorbar()
    Q = plt.quiver(X, Y, U, V, units='width')
    qk = plt.quiverkey(Q, 0.1, 0.1, 1, r'$spin$', labelpos='E',
                coordinates='figure')
    plt.title('T=%.2f'%temp+', #spins='+str(N)+'x'+str(N))
    plt.savefig(f'./images/{name}.png', dpi = 800)
    plt.close()

In [141]:
temp = [0.01, 0.05, 0.08, 0.1]
N = [32]

In [142]:
for n in tqdm(range(len(N))):    
    for i in range(len(temp)):
        xy1 = XYSystem(temperature = temp[i], width = N[n])
        xy1.equilibrate(show=False)
        x = list2matrix(xy1.spin_config)
        cu = get_curl(x, 1)
        plot_heatmap(x, cu, temp[i], f'mix2_{temp[i]}_{N[n]}')
        plot_heatmap(x, cu, temp[i], f'vec2_{temp[i]}_{N[n]}', False)
        plot_heatmap(x, cu, temp[i], f'curl2_{temp[i]}_{N[n]}', False, True)
        plot_heatmap(x, cu, temp[i], f'spin2_{temp[i]}_{N[n]}', False, False, True)
        print(f'max = {np.max(x)}, min = {np.min(x)}')

  0%|          | 0/1 [00:00<?, ?it/s]


equilibrium state is reached at T=0.0
#sweep=501
energy=-1.90
max = 12.899024602205733, min = -5.363957145045575

equilibrium state is reached at T=0.1
#sweep=502
energy=-1.90
max = 12.407062955897118, min = -5.681065585851291

equilibrium state is reached at T=0.1
#sweep=507
energy=-1.91
max = 13.082564093267905, min = -6.161455037272704

equilibrium state is reached at T=0.1
#sweep=508
energy=-1.92


100%|██████████| 1/1 [00:42<00:00, 42.05s/it]

max = 12.710600127579118, min = -7.119754885561425


In [143]:
cos(13.337968261936897)

0.716797575080995

In [144]:
x[0]

array([ 5.6024268 , -0.67810356,  5.86579671,  5.76903224,  6.25507732,
        6.05262184,  5.95486781,  5.99084723,  6.13874279,  0.26992914,
        0.07019746,  0.07817304,  6.08128966, -0.44714043,  6.16235063,
        6.01035176,  6.17004789, -0.69382886,  5.95818924,  5.67018139,
        5.60746874,  5.44297999,  5.60538382, 11.95838874,  5.12515146,
        5.36630947, 11.65894368,  4.91678201,  4.9524995 ,  5.45547448,
        5.28400854, -0.68063624])

In [145]:
cos(x[0])

array([0.77709555, 0.77876379, 0.91415065, 0.87070955, 0.999605  ,
       0.97353778, 0.94658621, 0.95757268, 0.9895863 , 0.96378979,
       0.99753717, 0.99694604, 0.97968821, 0.90168723, 0.99270837,
       0.96301123, 0.99360679, 0.76880316, 0.94765198, 0.81792347,
       0.78025897, 0.66730992, 0.77895325, 0.82080247, 0.40114133,
       0.60830279, 0.61577516, 0.20297286, 0.23780998, 0.67656324,
       0.54099485, 0.7771725 ])

In [146]:
cos(-1.88248406)

-0.3066654861976086